In [13]:
%matplotlib qt5
import sys
sys.path.append('./pdeSolver')

from domains import *
from equations import *
from torch_solvers import *
import torch

import ipywidgets as widgets
from ipywidgets import interact
from mpl_toolkits import mplot3d
import matplotlib.patches as patches
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt

In [2]:
dom=EmptyRoom(1.0,0.4,0.6)
#dom.plot_N_brownian_paths(np.sqrt(0.05),0.01,2000,0.0,[0.1,0.1],1,False,False)

In [3]:
eqn_config={"N":1,"nu":0.05,"lam":4.0}
eqn=HJB_LQR_Equation_2D(dom,eqn_config)

In [4]:
def l_schedule(epoch):
    return 1.0

solver_params={"initial_lr":0.01,
               "lambda_lr":l_schedule,
               "initial_loss_weigths":[1.0,1.0,1.0,1.0],
               "logging_interval":100,
               "dtype":torch.float32}

sol=DGM_solver(eqn,solver_params)

torch.float32


In [5]:
sol.train(3000)

Epoch  1  time  0.07319355010986328  loss  3.9819326  L1: 117.30921936035156  L2: 1.2018344402313232  L3:  2.872598171234131  L4:  2.5157036781311035
Epoch  101  time  2.5518555641174316  loss  0.27909583  L1: 0.0031471983529627323  L2: 0.23794256150722504  L3:  0.0019814406987279654  L4:  0.03626745566725731
Epoch  201  time  4.852170705795288  loss  0.2746056  L1: 0.0022755793761461973  L2: 0.2480759471654892  L3:  0.0015188611578196287  L4:  0.029721852391958237
Epoch  301  time  7.154136896133423  loss  0.2646704  L1: 0.0030639495234936476  L2: 0.24401287734508514  L3:  0.00179590517655015  L4:  0.020811446011066437
Epoch  401  time  9.462674140930176  loss  0.22763555  L1: 0.004178677685558796  L2: 0.2209652066230774  L3:  0.0019375698175281286  L4:  0.0064244456589221954
Epoch  501  time  11.759897470474243  loss  0.22233337  L1: 0.0028046825900673866  L2: 0.21329201757907867  L3:  0.001505075371824205  L4:  0.0050781890749931335
Epoch  601  time  14.077874183654785  loss  0.2133

array([[0.00000000e+00, 3.98193264e+00, 7.31935501e-02],
       [1.00000000e+02, 2.79095829e-01, 2.55185556e+00],
       [2.00000000e+02, 2.74605602e-01, 4.85217071e+00],
       [3.00000000e+02, 2.64670402e-01, 7.15413690e+00],
       [4.00000000e+02, 2.27635548e-01, 9.46267414e+00],
       [5.00000000e+02, 2.22333372e-01, 1.17598975e+01],
       [6.00000000e+02, 2.13319913e-01, 1.40778742e+01],
       [7.00000000e+02, 2.18467429e-01, 1.66272521e+01],
       [8.00000000e+02, 2.14497924e-01, 1.88040154e+01],
       [9.00000000e+02, 2.20293626e-01, 2.08955691e+01],
       [1.00000000e+03, 2.19190031e-01, 2.29459772e+01],
       [1.10000000e+03, 2.14206442e-01, 2.50114226e+01],
       [1.20000000e+03, 2.16677666e-01, 2.70316265e+01],
       [1.30000000e+03, 2.23350286e-01, 2.90669606e+01],
       [1.40000000e+03, 2.26778924e-01, 3.10979586e+01],
       [1.50000000e+03, 2.13574067e-01, 3.31683156e+01],
       [1.60000000e+03, 2.20662445e-01, 3.52239585e+01],
       [1.70000000e+03, 2.19111

In [17]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
x = y = np.arange(-0.05, 1.05, 0.05)
X, Y = np.meshgrid(x, y)
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_xlim(0,1)
ax.set_ylim(0,1)
ax.set_zlim(-5,5)

fig.subplots_adjust(bottom=0.25)

axfreq = fig.add_axes([0.25, 0.1, 0.65, 0.03])
freq_slider = Slider(
    ax=axfreq,
    label='t',
    valmin=0.0,
    valmax=1.0,
    valinit=0.0,
    valstep=0.01
)
draw(0)
def draw(t):  
    ax.cla()
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.set_zlim(-2,2)
    times=t*np.ones(np.ravel(X).shape[0])
    tes=torch.tensor(np.stack((times,np.ravel(X), np.ravel(Y)),axis=1),dtype=sol.dtype)
    zs =sol.model(tes).detach().numpy()
    Z = zs.reshape(X.shape)
    ax.plot_surface(X, Y, Z)
freq_slider.on_changed(draw)
#interact(draw,t=widgets.FloatSlider(min=0, max=1.0, step=0.01, value=10))


0